In [1]:
import pandas as pd
import pathlib
import polyflexmd.data_analysis.data.constants as data_constants
import polyflexmd.data_analysis.transform.transform as transform
import polyflexmd.data_analysis.theory.kremer_grest as kremer_grest
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
PATH_DF_TRAJECTORY_PROCESSED = pathlib.Path("/home/egor/Projects/polyflexmd/data/trajectories_test.csv")

In [4]:
df_traj = pd.read_csv(
        PATH_DF_TRAJECTORY_PROCESSED,
        dtype={
            **data_constants.RAW_TRAJECTORY_DF_COLUMN_TYPES,
            "molecule-ID": np.ushort,
            "kappa": "category",
        }
    )
df_traj

,t,id,type,x,y,z,molecule-ID,kappa
0,450010000,1,1,0.000000,0.000000,0.000000,1,190.2
1,450010000,2,1,-0.336178,0.186169,0.890632,1,190.2
2,450010000,3,2,-0.516345,0.398859,1.792500,1,190.2
3,450010000,4,2,-0.682514,0.663554,2.739000,1,190.2
4,450010000,5,2,-0.715175,0.949343,3.693260,1,190.2
...,...,...,...,...,...,...,...,...
27199995,535000000,3196,2,-16.042000,33.235500,-39.390400,50,190.2
27199996,535000000,3197,2,-15.760300,33.960800,-39.969101,50,190.2
27199997,535000000,3198,2,-15.532800,34.590302,-40.663101,50,190.2
27199998,535000000,3199,2,-15.251300,35.347000,-41.266499,50,190.2


In [10]:
N_beads = 64
kappa = 190.2
l_b = .97

In [11]:
l_k_theory = kremer_grest.bare_kuhn_length(kappa, l_b)
l_k_theory

368.018

In [14]:
l_K_estimate = transform.estimate_kuhn_length_df(
    df_trajectory=df_traj.loc[df_traj["molecule-ID"] < 10],
    group_by_params=["kappa"],
    N_beads=N_beads,
    l_b=l_b,
    n_processes=2
)
l_K_estimate

,l_K,d_l_K
kappa,,
190.2,382.772696,0.40718
